In [93]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

import statsmodels.api as sm
import scipy.stats as st

import warnings
warnings.filterwarnings("ignore")

In [118]:
da = pd.read_csv('week2/nhanes_2015_2016.csv')
da.head()

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
0,83732,1.0,NaN,1.0,1,1,62,3,1.0,5.0,...,124.0,64.0,94.8,184.5,27.8,43.3,43.6,35.9,101.1,2.0
1,83733,1.0,NaN,6.0,1,1,53,3,2.0,3.0,...,140.0,88.0,90.4,171.4,30.8,38.0,40.0,33.2,107.9,NaN
2,83734,1.0,NaN,NaN,1,1,78,3,1.0,3.0,...,132.0,44.0,83.4,170.1,28.8,35.6,37.0,31.0,116.5,2.0
3,83735,2.0,1.0,1.0,2,2,56,3,1.0,5.0,...,134.0,68.0,109.8,160.9,42.4,38.5,37.7,38.3,110.1,2.0
4,83736,2.0,1.0,1.0,2,2,42,4,1.0,4.0,...,114.0,54.0,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2.0


In [120]:
da["SMQ020x"] = da.SMQ020.replace({1: "Yes", 2: "No", 7: np.nan, 9: np.nan})
da["RIAGENDRx"] = da.RIAGENDR.replace({1: "Male", 2: "Female"})
dx = da[["SMQ020x", "RIAGENDRx","BMXBMI"]].dropna()
dx.head()

,SMQ020x,RIAGENDRx,BMXBMI
0,Yes,Male,27.8
1,Yes,Male,30.8
2,Yes,Male,28.8
3,No,Female,42.4
4,No,Female,20.3


# Statistical Inference with Confidence Intervals
- 
<font color='red'>
\begin{align} Estimate &\pm t_{\nu/2} * SE \nonumber\\
SE =& \sqrt{\frac{p(1-p)}{n}} ~~~~ (Proportion)\nonumber\\
SE =& \frac{\sigma}{\sqrt{n}} ~~~~~~~~~~~~~~~~~ (Mean) \nonumber\\
SE =& \sqrt{SE_1^2+SE_2^2} ~~~~ (Two~populations~Difference)\end{align}</font>

## Why Confidence Intervals?

**Confidence intervals** are a calculated range or boundary around a parameter or a statistic that is supported mathematically with a certain level of confidence.
- 95% confidence level refers to our **confidence in the statistical procedure** that was used to make this interval.

## How to Calculate Confidence Intervals?
<font color='blue'>$$Best\ Estimate \pm Margin\ of\ Error$$</font>

- Where the *Best Estimate* is the **observed population proportion or mean** and the *Margin of Error* is the **t_multiplier**.

- The t-multiplier is calculated based on the degrees of freedom and desired confidence level.  
For samples with more than 30 observations and a confidence level of 95%, the t-multiplier is 1.96.

The equation to create a 95% confidence interval can also be shown as:
<font color='blue'>$$Population\ Proportion\ or\ Mean\ \pm (t\_multiplier *\ Standard\ Error)$$</font>

- The Standard Error is calculated differently for population proportion and mean:
$$Standard\ Error \ for\ Population\ Proportion = \sqrt{\frac{Population\ Proportion * (1 - Population\ Proportion)}{Number\ Of\ Observations}} = \sqrt{\frac{p(1-p)}{n}}$$

$$Standard\ Error \ for\ Mean = \frac{Standard\ Deviation}{\sqrt{Number\ Of\ Observations}} = \frac{\sigma}{\sqrt{n}}$$

### Confidence intervals for one proportion

A sample of 659 parents with a toddles was taken and asked if they used a car seat for all travel with their toddler. 559 parents reponded Yes to this question. Construct a 95% confidence interval for the population proportion of parents reporting they use a car seat for all travel with their toddler.

In [24]:
t = 1.96
p = 559/659
n = 659
se = np.sqrt(p*(1-p)/n)

lb = p - t*se
ub = p + t*se
print('between',lb,'and',ub)

between 0.8208622940192611 and 0.8756475694101773


In [25]:
sm.stats.proportion_confint(559, 659,alpha = 0.05)

(0.8208627973654069, 0.8756470660640315)

- We estimated, with 95% confidence, that the population proportion of parents with a toddler that use a car seat for all travel with their toddler was somewhere between 82.2% and 87.7%.
- Essentially, if we repeat this process, **95% of our calculated confidence intervals would contain the true proportion**.

In [17]:
dz1 = dx.groupby(dx.RIAGENDRx).agg({"SMQ020x": [lambda x: np.mean(x=="Yes"), np.size]})
dz1.columns = ["Proportion", "Total_n"] # The default column names are unclear, so we replace them here
dz1

,Proportion,Total_n
RIAGENDRx,,
Female,0.304845,2972
Male,0.513258,2753


In [21]:
p = dz1.Proportion.Male # Male proportion
n = dz1.Total_n.Male # Total number of males
lcb = p - 1.96 * np.sqrt(p * (1 - p) / n)  
ucb = p + 1.96 * np.sqrt(p * (1 - p) / n)  
print(lcb, ucb)

0.49458714955108174 0.531929377873546


In [23]:
# 95% CI for the proportion of males who smoke (compare to value above)
sm.stats.proportion_confint(1413, 1413+1340) 

(0.49458749263718593, 0.5319290347874418)

In [20]:
p = dz1.Proportion.Female # Female proportion
n = dz1.Total_n.Female # Total number of females
lcb = p - 1.96 * np.sqrt(p * (1 - p) / n)  
ucb = p + 1.96 * np.sqrt(p * (1 - p) / n)  
print(lcb, ucb)

0.288294683866098 0.32139576027925865


In [22]:
# 95% CI for the proportion of females who smoke (compare to value above)
sm.stats.proportion_confint(906, 906+2066)

(0.2882949879861214, 0.32139545615923526)

The results above indicate that any population proportion (for female lifetime smokers) between 0.288 and 0.321 would be compatible with the data that we observed in NHANES.

### Mean confidence interval:

In [ ]:
# Example 1
df = pd.read_csv("week2/Cartwheeldata.csv")

In [ ]:
df.head()

In [ ]:
mean = df["CWDistance"].mean()
sd = df["CWDistance"].std()
n = len(df)

t = 2.064
se = sd/np.sqrt(n)

lcb = mean - t * se
ucb = mean + t * se
(lcb, ucb)

In [ ]:
sm.stats.DescrStatsW(df["CWDistance"]).zconfint_mean()

In [ ]:
# Example 2

In [26]:
da.groupby("RIAGENDRx").agg({"BMXBMI": [np.mean, np.std, np.size]})

BMXBMI                  
                mean       std    size
RIAGENDRx                             
Female     29.939946  7.753319  2976.0
Male       28.778072  6.252568  2759.0

In [27]:
lcb_female = 29.94 - 1.96 * 7.753 / np.sqrt(2976)
ucb_female = 29.94 + 1.96 * 7.753 / np.sqrt(2976)
print(lcb_female, ucb_female)

29.661446004723665 30.218553995276338


In [28]:
female_bmi = da.loc[da.RIAGENDRx=="Female", "BMXBMI"].dropna()
sm.stats.DescrStatsW(female_bmi).zconfint_mean()

(29.659875498090155, 30.22001580625768)

In [117]:
mean = 62.5
sd = 10
n = 500

z = 1.96
se = sd/np.sqrt(n)

lcb = mean - z * se
ucb = mean + z * se
(lcb, ucb)

(61.62346135282008, 63.37653864717992)

## Difference of Two Population
The standard error for difference of population proportions and means is:

$$Standard\ Error\ for\ Difference\ of\ Two\ Population\ Proportions\ Or\ Means = \sqrt{SE_{Proportion\ 1}^2 + SE_{Proportion\ 2} ^2}$$

### Difference of Two Population Proportions
Analyze the difference of proportion between female and male smokers "SMQ020". 

In [6]:
pd.crosstab(dx.SMQ020x, dx.RIAGENDRx)

RIAGENDRx,Female,Male
SMQ020x,,
No,2066,1340
Yes,906,1413


In [4]:
dz = dx.groupby("RIAGENDRx").agg({"SMQ020x": [np.size]})
dz.column = ["Proportion", "Total n"]
dz

,SMQ020x
,size
RIAGENDRx,
Female,2972
Male,2753


In [8]:
p = 906/2972
n = 2972
se_female = np.sqrt(p * (1 - p)/n)
se_female

0.008444152146214435

In [9]:
p = 1413/2753
n = 2753
se_male = np.sqrt(p * (1 - p)/ n)
se_male

0.009526078653689868

In [10]:
se_diff = np.sqrt(se_female**2 + se_male**2)

d = 906/2972 - 1413/2753
lcb = d - 1.96 * se_diff
ucb = d + 1.96 * se_diff
(lcb, ucb)

(-0.2333636091471941, -0.18346247413207697)

### Difference of Two Population Mean

Analyze the difference of mean of body mass index within our female and male populations "BMXBMI". 

In [11]:
da["BMXBMI"].head()

0    27.8
1    30.8
2    28.8
3    42.4
4    20.3
Name: BMXBMI, dtype: float64

In [12]:
da.groupby("RIAGENDRx").agg({"BMXBMI": [np.mean, np.std, np.size]})

BMXBMI                  
                mean       std    size
RIAGENDRx                             
Female     29.939946  7.753319  2976.0
Male       28.778072  6.252568  2759.0

In [13]:
sem_female = 7.753319 / np.sqrt(2976)
sem_male = 6.252568 / np.sqrt(2759)
(sem_female, sem_male)

(0.14212523289878048, 0.11903716451870151)

In [16]:
sem_diff = np.sqrt(sem_female**2 + sem_male**2)
d = 29.939946 - 28.778072
lcb = d - 1.96 * sem_diff
ucb = d + 1.96 * sem_diff
(lcb, ucb)

(0.798509725476467, 1.5252382745235278)

## Sample size
$\displaystyle p\pm z^*\sqrt{\frac{p(1-p)}{n}}$, when $p=1/2$, consevative Standard Error $\displaystyle=\frac{1}{2\sqrt{n}}$, for 95% confidence level, $z^*=1.96$, then margin of error is $\displaystyle\frac{1}{\sqrt{n}}$.
- Margin of Error (MoE) is only dependent on 
    - confidence level (typically 95%) 
    - sample size

<font color='red'>$\displaystyle \ n = (\frac{z^*}{2MoE})^2$</font>
- Estimated standard error may be too small, or inaccurate based of sample so can employ conservative approach
- conservative approach, determine sample size needed based on a confidence level and desired margin of error

Whar sample size would you need to have a 95% conservation confidence interval with a Margin of Error of only 4%?
n = (1.96/(2*0.04))**2, n=600.25

In [29]:
n = (1.96/(2*0.04))**2
n

600.25

# Hypothesis Testing
**Null Hypothesis: $H_0$**

**Alternative Hypothesis: $H_a$**
<font color='red'>
\begin{align} \displaystyle
    z\_score = &\frac{Best\ Estimate - Hypothesized\ Estimate}{Standard\ Error\ of\ Estimate}
\end{align}</font>
One can calculate the P_value with z_score. If P_value < 0.05, then rejecte H0.
    
## One Population Proportion

In previous years 52% of parents believed that electronics and social media was the cause of their teenager’s lack of sleep. Do more parents today believe that their teenager’s lack of sleep is caused due to electronics and social media? 

**Population**: Parents with a teenager (age 13-18)  
**Parameter of Interest**: p  
**Null Hypothesis:** p = 0.52  
**Alternative Hypthosis:** p > 0.52  

Today, 1018 Parents, 56% believe that their teenager’s lack of sleep is caused due to electronics and social media.

In [95]:
n = 1018
pnull = .52
phat = .56
sm.stats.proportions_ztest(phat * n, n, pnull)

(2.571067795759113, 0.010138547731721065)

In [96]:
z_score = (0.56-0.52)/np.sqrt(0.56*(1-0.56)/n)
p_values = st.norm.sf(abs(z_score))*2
(z_score,p_values)

(2.5710677957591126, 0.010138547731721084)

- **Z=2.57** That means that our observed sample proportion is 2.55 null standard errors above our hypothesized population proportion.
- **P_value=0.01** < $\alpha = 0.05$
- Reject the null hypothesis. 
- There is sufficient evidence to conclude that the population proportion of parents with a teenager who belive that electronics and social media is the cause for lack of sleep is greater than 52%.

## Difference in Population Proportions

247 Parents of Black Children
36.8% of parents report that their child has had some swimming lessons.
308 Parents of Hispanic Children
38.9% of parents report that their child has had some swimming lessons.

Is there a **significant difference** between the population proportions of parents of black children and parents of Hispanic children who report that their child has had some swimming lessons?

**Populations**: All parents of black children age 6-18 and all parents of Hispanic children age 6-18  
**Parameter of Interest**: p1 - p2, where p1 = black and p2 = hispanic  

**Null Hypothesis:** p1 - p2 = 0  , **no significant difference**

**Alternative Hypthosis:** p1 - p2 $\neq$ 0  ,**has significant difference**

<font color='red'> $\displaystyle z\_score = \frac{p-p_0}{SE(p)}$    
SE is the standard error of estimate $\displaystyle SE = \sqrt{p(1-p)(\frac{1}{n_1}+\frac{1}{n_2})} $

In [101]:
p1 = 0.37
n1 = 247

p2 = 0.39
n2 = 308

p12 = p2-p1
sep =np.sqrt(p1*(1-p1)*(1.0/n1+1.0/n2)) 
z_score1 = (p1-p2)/sep
p_value1 = st.norm.sf(abs(z_score1))*2 

print('z_score:',z_score1,'Pvalue:',p_value1)

sep: 0.04123763909554543
z_score: -0.4849938172663346 Pvalue: 0.6276807312529116


- P_value=0.63 > 0.1. 
- Formally, based on our sample and our p-value, we fail to reject the null hypothesis. 
- We conclude that there is **no significant difference** between the population proportion of parents of black and Hispanic children.

### Chi-Square Test

In [108]:
tb1 = pd.DataFrame([[91,156],[120,188]])
tb1.columns = ['Yes','No']
tb1

,Yes,No
0,91,156
1,120,188


In [114]:
# 看一下期望频数
st.contingency.expected_freq(tb1)

array([[ 93.9045045, 153.0954955],
       [117.0954955, 190.9045045]])

In [115]:
st.chi2_contingency(tb1,False)

(0.26117673621545673, 0.6093128715165168, 1, array([[ 93.9045045, 153.0954955],
        [117.0954955, 190.9045045]]))

- chi2 = 0.26, Pvalue = 0.61, 不能拒绝H0，no siginificant difference,degree of freedom = 1, 
- 期望频数（expected frequency）

## One Population Mean

Is the average cartwheel distance (in inches) for adults 
more than 80 inches?

**Population**: All adults  
**Parameter of Interest**: $\mu$, population mean cartwheel distance.

**Null Hypothesis:** $\mu$ = 80

**Alternative Hypthosis:** $\mu$ > 80

25 Adults

$\mu = 82.46$

$\sigma = 15.06$

In [85]:
mu = 82.46
mu0 = 80
sigma = 15.06
n = 25
z = (mu-mu0)/(sigma/np.sqrt(n))
p = st.norm.sf(abs(z)) 
print('z_score:',z,'Pvalue:',p)

z_score: 0.8167330677290816 Pvalue: 0.20704049479973047


In [86]:
df = pd.read_csv("week2/Cartwheeldata.csv")
n = len(df)
mean = df["CWDistance"].mean()
sd = df["CWDistance"].std()
(n, mean, sd)

(25, 82.48, 15.058552387264855)

In [87]:
sm.stats.ztest(df["CWDistance"], value = 80, alternative = "larger")

(0.8234523266982029, 0.20512540845395266)

## Two Population Means Difference 

Considering adults in the NHANES data, do males have a significantly higher mean Body Mass Index than females?

**Population**: Adults in the NHANES data.  
**Parameter of Interest**: $\mu_1 - \mu_2$, Body Mass Index.  
**Null Hypothesis:** $\mu_1 = \mu_2$  
**Alternative Hypthosis:** $\mu_1 \neq \mu_2$

2976 Females 
$\mu_1 = 29.94$  
$\sigma_1 = 7.75$  

2759 Male Adults  
$\mu_2 = 28.78$  
$\sigma_2 = 6.25$  

$\mu_1 - \mu_2 = 1.16$

In [89]:
n1 = 2976
mu1 = 29.94
sigma1 = 7.75

n2 = 2759
mu2 = 28.78
sigma2 = 6.25

zs = (mu1-mu2)/(np.sqrt(sigma1**2/n1+sigma2**2/n2))
ps = st.norm.sf(abs(zs)) 
print('z_score:',zs,'Pvalue:',ps)

z_score: 6.259716645914027 Pvalue: 1.9283879867592308e-10


In [116]:
females = da[da["RIAGENDR"] == 2]
male = da[da["RIAGENDR"] == 1]

n1 = len(females)
mu1 = females["BMXBMI"].mean()
sd1 = females["BMXBMI"].std()

n2 = len(male)
mu2 = male["BMXBMI"].mean()
sd2 = male["BMXBMI"].std()

zs = (mu1-mu2)/(np.sqrt(sd1**2/n1+sd2**2/n2))
ps = st.norm.sf(abs(zs)) 
print('z_score:',zs,'Pvalue:',ps)

z_score: 6.267188023490217 Pvalue: 1.838130807926842e-10


In [92]:
sm.stats.ztest(females["BMXBMI"].dropna(), male["BMXBMI"].dropna())

(6.1755933531383205, 6.591544431126401e-10)